In [1]:
import sklearn

#NN Surrogate model class
from injector_surrogate_quads import *

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#BO
from bayes_opt import BayesianOptimization

# Load injector model

In [2]:
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

# Set up sampling and objectives

In [9]:
#convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

#input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale', 'SQ01:b1_gradient','CQ01:b1_gradient'] #

#output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def evaluate(varx,vary,varz): 

    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    #fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    #set solenoid, SQ, CQ to values from optimization step
    x_in[:, Model.loc_in[opt_var_names[0]]] = varx
    x_in[:, Model.loc_in[opt_var_names[1]]] = vary
    x_in[:, Model.loc_in[opt_var_names[2]]] = varz

    #output predictions
    y_out = Model.pred_machine_units(x_in) 

    return -1*objective(y_out)[0]


def objective(y_out):
    
    #output is geometric emittance in transverse plane
    out1 = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
       
    return np.sqrt(out1*out2)/1e-6 # in um units

# Simple BO

In [10]:
# bounds on input params 
pbounds = {'varx': (0.44, 0.55),
           'vary': (-0.02, 0.02),
           'varz': (-0.02, 0.02)
          }

optimizer = BayesianOptimization(
    f = evaluate,
    pbounds = pbounds,
    random_state = 1,
)

optimizer.maximize(
    init_points=5,
    n_iter=40,
)

|   iter    |  target   |   varx    |   vary    |   varz    |
-------------------------------------------------------------
|  1        | -1.284e+0 |  0.4859   |  0.008813 | -0.02     |
|  2        | -1.284e+0 |  0.4733   | -0.01413  | -0.01631  |
|  3        | -1.284e+0 |  0.4605   | -0.006178 | -0.004129 |
|  4        | -1.284e+0 |  0.4993   | -0.003232 |  0.007409 |
|  5        | -1.284e+0 |  0.4625   |  0.01512  | -0.0189   |
|  6        | -1.284e+0 |  0.4813   |  0.0116   |  0.01098  |
|  7        | -1.284e+0 |  0.4986   |  0.0115   | -0.013    |
|  8        | -1.284e+0 |  0.5055   | -0.009559 |  0.009869 |
|  9        | -1.284e+0 |  0.465    | -0.01378  |  0.01789  |
|  10       | -1.284e+0 |  0.4732   | -0.0146   | -0.01722  |
|  11       | -1.284e+0 |  0.4917   | -0.00306  | -0.001123 |
|  12       | -1.284e+0 |  0.5214   |  0.000749 |  0.0198   |
|  13       | -1.284e+0 |  0.4995   | -0.003835 |  0.007813 |
|  14       | -1.284e+0 |  0.4755   |  0.01054  |  0.01428  |
|  15   

## Results from simple BO

In [7]:
SOL_opt = optimizer.max['params']['varx'] # solenoid val at optimum
CQ_opt = optimizer.max['params']['vary'] # CQ val at optimum
SQ_opt = optimizer.max['params']['varz'] # SQ val at optimum

opt_emit = -1*optimizer.max['target'] # emittance value at optimum (in um)

print('optimum (pv_units) ',SOL_opt, CQ_opt, SQ_opt)
print('optimum geom emit ', opt_emit)

optimum (pv_units)  0.47737014691426854 -0.0004981337721706551 0.0004994750148957644
optimum geom emit  0.5750189423561096
